<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=89c9e8e1ef58afe7c74df9f4e4cae86134493f5036c62fc626590237af23b6d9
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65


    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-15 11:17:59
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.18 L
Current:  1.41 C
-------------------
Today PnL: -9.15 K (-0.06%)
Current PnL: -17.71 L (-11.91%)
CY Booked + Current PnL: -6.68 L (-4.5%)
-------------------
Total profit:  2.05 L
Total loss:  -19.76 L
-------------------
Total Booked + Current PnL: 20.30 L (16.68%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.8%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 83.82 L (59.24%)
Deployed:  1.22 C
Current:  1.41 C
CAGR/XIRR %: 9.37%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.75,57.0,H-LC,11.14,172392.0,16215.0,9878.0,-0.49,10.38,5.73,16.71,22.0,1.64,1.26,29.63,X40N,BTT,HEALTHCARE
62,SBIN,863.00,-11.34,61.0,M-LC,4.25,218386.0,16906.0,10308.0,0.07,8.39,4.72,13.51,60.0,1.64,1.59,22.05,XY25,NTT,BANKS
77,TTKPRESTIG,770.00,104.70,59.0,M-SC,4.71,88215.0,-12562.0,12659.0,0.51,-12.47,14.35,0.09,245.0,-0.99,0.64,15.90,OX40N,NTT,DURABLES
49,NATIONALUM,247.44,-41.18,75.0,H-MC,5.14,116929.0,16425.0,14219.0,1.06,16.34,12.16,30.49,79.0,1.16,0.85,54.04,MH,ATH,METALS
41,ITC,452.00,-37.50,53.0,H-LC,2.91,201859.0,1721.0,19177.0,-0.23,0.86,9.50,10.44,4.0,0.09,1.47,6.47,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.00,-20.65,44.0,H-MC,2.19,136448.0,660.0,42094.0,0.49,0.49,30.85,31.48,107.0,0.02,0.99,12.66,X40,ATH,INSURANCE
30,HAVELLS,2069.16,0.06,61.0,H-MC,1.78,247915.0,184.0,74870.0,1.06,0.07,30.20,30.30,92.0,0.00,1.81,13.38,X40,ATH,ELECTRICAL
12,BANKINDIA,190.00,-30.43,61.0,H-MC,5.74,180792.0,984.0,111422.0,0.18,0.55,61.63,62.52,88.0,0.01,1.32,34.82,XR,NTT,BANKS


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-16.13,39.0,H-LC,5.80,301870.0,-13972.0,126634.0,0.71,-4.42,41.95,35.67,5.0,-0.11,2.20,8.82,X40N,ATH,FMCG
43,JIOFIN,387.00,-1.80,42.0,H-LC,13.42,266815.0,-550.0,66010.0,-0.31,-0.21,24.74,24.48,48.0,-0.01,1.94,54.65,XY24,BTT,FINANCE
31,HCLTECH,1943.91,-0.22,45.0,H-LC,10.10,228247.0,-13677.0,76942.0,-0.91,-5.65,33.71,26.15,8.0,-0.18,1.66,7.86,X40,ATH,IT
80,UNITDSPR,1644.00,-14.09,47.0,H-LC,8.05,225268.0,-8876.0,57488.0,-0.71,-3.79,25.52,20.77,86.0,-0.15,1.64,2.87,X40N,NTT,BREWERIES
6,ASIANPAINT,4250.00,-12.54,48.0,H-LC,8.42,220669.0,-31099.0,153321.0,-1.52,-12.35,69.48,48.55,27.0,-0.20,1.61,18.70,X40,BTT,PAINTS


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-3.76,57.0,H-LC,1.09,238500.0,-7651.0,86862.0,-0.61,-3.11,36.42,32.18,16.0,-0.09,1.74,31.81,X200,ATH,IT
41,ITC,452.00,-37.50,53.0,H-LC,2.91,201859.0,1721.0,19177.0,-0.23,0.86,9.50,10.44,4.0,0.09,1.47,6.47,X40,NTT,FMCG
50,NESTLEIND,1377.00,-7.71,60.0,H-LC,4.13,284076.0,18650.0,38151.0,-0.30,7.03,13.43,21.40,11.0,0.49,2.07,15.36,XY25,NTT,FMCG
83,VBL,671.64,-16.13,39.0,H-LC,5.80,301870.0,-13972.0,126634.0,0.71,-4.42,41.95,35.67,5.0,-0.11,2.20,8.82,X40N,ATH,FMCG
56,RELIANCE,1533.00,-13.29,52.0,H-LC,5.85,217136.0,6770.0,22018.0,-0.22,3.22,10.14,13.68,37.0,0.31,1.58,20.27,XY25,NTT,REFINERIES


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TITAGARH,1548.00,1.84,71.0,H-SC,4.92,167116.0,-24899.0,103779.0,2.96,-12.97,62.10,41.08,158.0,-0.24,1.22,41.61,XY24,NTT,ENGINEERING
66,SIS,528.00,2160.10,46.0,H-SC,5.38,91214.0,-19818.0,43427.0,2.70,-17.85,47.61,21.26,156.0,-0.46,0.66,23.24,OX40N,NTT,MISC
3,ALKYLAMINE,4546.37,-4.67,46.0,H-SC,13.68,88649.0,-12314.0,106849.0,1.53,-12.20,120.53,93.63,148.0,-0.12,0.65,32.04,SR,ATH,CHEMICALS
53,RAJESHEXPO,518.00,1850.76,64.0,L-SC,4.98,53233.0,-83944.0,84039.0,1.50,-61.19,157.87,0.07,267.0,-1.00,0.39,32.39,OX40N,NTT,JEWELLERY
1,ABB,7934.00,-38.80,61.0,H-LC,11.54,255629.0,-5990.0,125207.0,1.47,-2.29,48.98,45.57,7.0,-0.05,1.86,8.77,AR,NTT,ELECTRICAL


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.00,3797.30,62.0,M-SC,6.76,121410.0,-25958.0,77775.0,-1.95,-17.61,64.06,35.16,236.0,-0.33,0.88,24.01,XY24,NTT,AUTO
48,MASFIN,398.61,-16.44,55.0,H-SC,12.82,95085.0,-2895.0,24494.0,-1.80,-2.95,25.76,22.05,152.0,-0.12,0.69,38.21,XR,ATH,FINANCE
7,ASIANTILES,137.00,6488.89,40.0,L-SC,5.66,72289.0,-21521.0,98140.0,-1.74,-22.94,135.76,81.67,269.0,-0.22,0.53,39.99,XR,NTT,CERAMICS
6,ASIANPAINT,4250.00,-12.54,48.0,H-LC,8.42,220669.0,-31099.0,153321.0,-1.52,-12.35,69.48,48.55,27.0,-0.20,1.61,18.70,X40,BTT,PAINTS
22,COFFEEDAY,80.00,-41.65,53.0,L-SC,45.57,85651.0,-27898.0,67947.0,-1.41,-24.57,79.33,35.27,268.0,-0.41,0.62,109.63,XR,NTT,HOTELS


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,98.37,59.0,H-SC,12.46,125577.0,-16902.0,36417.0,-0.24,-11.86,29.00,13.70,163.0,-0.46,0.91,48.22,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,104.70,59.0,M-SC,4.71,88215.0,-12562.0,12659.0,0.51,-12.47,14.35,0.09,245.0,-0.99,0.64,15.90,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.42,61.0,H-SC,1.78,224028.0,-45639.0,81972.0,0.56,-16.92,36.59,13.47,138.0,-0.56,1.63,13.31,XY24,NTT,PAINTS
19,CERA,9475.0,-20.29,44.0,H-SC,2.16,145337.0,-30566.0,72581.0,-0.05,-17.38,49.94,23.89,149.0,-0.42,1.06,25.52,OX40N,NTT,CERAMICS
66,SIS,528.0,2160.10,46.0,H-SC,5.38,91214.0,-19818.0,43427.0,2.70,-17.85,47.61,21.26,156.0,-0.46,0.66,23.24,OX40N,NTT,MISC


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-26.76,50.0,H-LC,13.13,291287.0,-54669.0,178530.0,-1.10,-15.80,61.29,35.80,1.0,-0.31,2.12,3.19,X40,BTT,IT
39,INFY,2275.00,-17.80,52.0,H-LC,9.14,320565.0,7677.0,164001.0,-1.35,2.45,51.16,54.87,3.0,0.05,2.33,9.27,X40,BTT,IT
41,ITC,452.00,-37.50,53.0,H-LC,2.91,201859.0,1721.0,19177.0,-0.23,0.86,9.50,10.44,4.0,0.09,1.47,6.47,X40,NTT,FMCG
83,VBL,671.64,-16.13,39.0,H-LC,5.80,301870.0,-13972.0,126634.0,0.71,-4.42,41.95,35.67,5.0,-0.11,2.20,8.82,X40N,ATH,FMCG
1,ABB,7934.00,-38.80,61.0,H-LC,11.54,255629.0,-5990.0,125207.0,1.47,-2.29,48.98,45.57,7.0,-0.05,1.86,8.77,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,45.15,66.0,M-MC,6.95,229009.0,4047.0,163604.0,0.55,1.80,71.44,74.53,192.0,0.02,1.67,34.63,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,9.32,53.0,H-MC,3.82,188640.0,23970.0,43991.0,0.08,14.56,23.32,41.28,89.0,0.54,1.37,23.70,X40N,ATH,AC
84,VOLTAS,1918.49,0.88,58.0,H-MC,2.83,211650.0,19908.0,76131.0,0.53,10.38,35.97,50.08,99.0,0.26,1.54,17.89,XY25,ATH,AC
30,HAVELLS,2069.16,0.06,61.0,H-MC,1.78,247915.0,184.0,74870.0,1.06,0.07,30.20,30.30,92.0,0.00,1.81,13.38,X40,ATH,ELECTRICAL
24,DABUR,735.00,-3.12,57.0,H-MC,6.39,210932.0,13904.0,75725.0,0.37,7.06,35.90,45.49,102.0,0.18,1.54,21.02,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-41.18,75.0,H-MC,5.14,116929.0,16425.0,14219.0,1.06,16.34,12.16,30.49,79.0,1.16,0.85,54.04,MH,ATH,METALS
36,INDIAMART,4850.92,-50.53,53.0,H-SC,12.21,137975.0,14639.0,119128.0,0.10,11.87,86.34,108.45,119.0,0.12,1.00,36.88,AR,ATH,MISC
85,WHIRLPOOL,2270.00,-40.79,51.0,M-SC,1.11,100065.0,8566.0,70186.0,-1.28,9.36,70.14,86.07,223.0,0.12,0.73,45.31,XR,NTT,DURABLES
86,WIPRO,420.00,-11.52,51.0,M-LC,5.71,155198.0,4253.0,105209.0,-0.64,2.82,67.79,72.51,53.0,0.04,1.13,9.01,XR,NTT,IT
12,BANKINDIA,190.00,-30.43,61.0,H-MC,5.74,180792.0,984.0,111422.0,0.18,0.55,61.63,62.52,88.0,0.01,1.32,34.82,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-41.65,53.0,L-SC,45.57,85651.0,-27898.0,67947.0,-1.41,-24.57,79.33,35.27,268.0,-0.41,0.62,109.63,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.67,46.0,H-SC,13.68,88649.0,-12314.0,106849.0,1.53,-12.20,120.53,93.63,148.0,-0.12,0.65,32.04,SR,ATH,CHEMICALS
48,MASFIN,398.61,-16.44,55.0,H-SC,12.82,95085.0,-2895.0,24494.0,-1.80,-2.95,25.76,22.05,152.0,-0.12,0.69,38.21,XR,ATH,FINANCE
49,NATIONALUM,247.44,-41.18,75.0,H-MC,5.14,116929.0,16425.0,14219.0,1.06,16.34,12.16,30.49,79.0,1.16,0.85,54.04,MH,ATH,METALS
82,VALIANTORG,838.00,-324.58,54.0,H-SC,7.44,124984.0,-38621.0,159942.0,0.74,-23.61,127.97,74.15,139.0,-0.24,0.91,60.21,XR,NTT,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-41.65,53.0,L-SC,45.57,85651.0,-27898.0,67947.0,-1.41,-24.57,79.33,35.27,268.0,-0.41,0.62,109.63,XR,NTT,HOTELS
49,NATIONALUM,247.44,-41.18,75.0,H-MC,5.14,116929.0,16425.0,14219.0,1.06,16.34,12.16,30.49,79.0,1.16,0.85,54.04,MH,ATH,METALS
8,ATULAUTO,844.00,3797.30,62.0,M-SC,6.76,121410.0,-25958.0,77775.0,-1.95,-17.61,64.06,35.16,236.0,-0.33,0.88,24.01,XY24,NTT,AUTO
75,TITAGARH,1548.00,1.84,71.0,H-SC,4.92,167116.0,-24899.0,103779.0,2.96,-12.97,62.10,41.08,158.0,-0.24,1.22,41.61,XY24,NTT,ENGINEERING
27,GREENPANEL,537.00,237.86,67.0,M-SC,4.73,150865.0,-32213.0,111188.0,-1.36,-17.60,73.70,43.14,230.0,-0.29,1.10,41.91,XY24,NTT,MISC


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.75
1,25,43.44
2,50,73.84


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.44
LC    34.44
MC    23.10
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.96
X40      15.21
X40N     12.97
XR       10.01
XY25      9.78
AR        8.62
OX40N     8.19
MH        1.74
X200      1.74
X5K       1.47
SR        1.29
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    27.39
H-LC    26.21
H-MC    20.13
M-SC    13.51
M-LC     7.17
M-MC     2.66
L-SC     1.54
L-LC     1.06
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.31,-15.06,80.48
FMCG,11.86,-4.03,40.42
FINANCE,9.60,-16.69,62.39
BANKS,7.76,-12.51,64.31
MISC,6.06,-14.05,79.16
PAINTS,5.87,-14.52,40.16
ELECTRICAL,5.65,-6.21,64.75
AC,3.62,2.75,30.19
INSURANCE,3.40,-7.70,44.65


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2868063.0,22
AR,1313202.0,9
XR,1238194.0,13
X40,877541.0,10
X40N,631020.0,10
OX40N,520964.0,10
XY25,408648.0,7
SR,245411.0,2
X5K,115157.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3353855.0,28
H-LC,1394143.0,15
M-SC,1389562.0,17
H-MC,1189420.0,15
M-LC,392716.0,5
M-MC,313094.0,2
L-SC,250126.0,3
L-MC,60427.0,1
L-LC,38345.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1036300.0      6
M-SC       XY24       802884.0      7
H-SC       AR         787091.0      5
           XR         678031.0      6
H-LC       X40        591971.0      5
H-MC       XY24       487733.0      4
H-LC       AR         363596.0      2
M-MC       XY24       313094.0      2
H-SC       X40N       298175.0      4
           OX40N      246440.0      4
           SR         245411.0      2
H-LC       X40N       225535.0      4
H-MC       X40        216476.0      4
           XY25       186044.0      2
L-SC       XR         166087.0      2
M-SC       AR         162515.0      2
M-LC       XY24       162042.0      2
M-SC       OX40N      124269.0      4
           XR         117018.0      2
M-LC       X5K        115157.0      1
M-SC       XY25       113782.0      1
H-MC       XR         111422.0      1
           X40N       107310.0      2
M-LC       XR         105209.0      1
H-LC       X200        86862.0      1
L-SC       OX40N       84039.0      1
M-SC       X40         69094.0      1
H-MC       OX40N       66216.0      1
H-LC       XY24        66010.0      1
H-SC       MH          62407.0      1
L-MC       XR          60427.0      1
H-LC       XY25        60169.0      2
L-LC       XY25        38345.0      1
H-MC       MH          14219.0      1
M-LC       XY25        10308.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 52.0 seconds
